In [123]:
## データセット作成

In [124]:
## インポート
import csv
import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder

In [125]:
%pwd

'/Users/user/scikit-learn'

In [126]:
## 1. custデータ読み込み

df_cust = pd.read_csv('./data/org/20181005.cust.csv', encoding='Shift_JISx0213')
df_cust[:5]

,更新日時,登録日時,無効,顧客ID,氏名,カナ,登録区分,ログインID,ログインパスワード,PW変更区分,...,不達,DM送付拒否,FBメール送付拒否,販促電話拒否,職業,勤務先,顧客注意事項,予約確認方法,コンタクト履歴,全店備考
0,2018-09-29 07:00:13,2008-08-22 19:36:15,NaN,000112093,NaN,NaN,本登録,000112093,NaN,NaN,...,NaN,携帯メール,携帯メール/PCメール,NaN,会社員,NaN,NaN,NaN,2018-09-28 16:00 【PCメール】10月CP-BLC メール送信/2018-0...,NaN
1,2018-09-29 07:00:13,2006-10-24 00:00:00,NaN,000121059,NaN,NaN,本登録,000121059,NaN,NaN,...,携帯メール,NaN,NaN,NaN,会社員,NaN,NaN,NaN,2018-09-28 16:00 【PCメール】10月CP-BLC メール送信/2018-0...,NaN
2,2018-09-29 07:00:13,2007-05-08 00:00:00,NaN,000321123,NaN,NaN,本登録,000321123,NaN,NaN,...,NaN,NaN,NaN,NaN,会社員,NaN,NaN,NaN,2018-09-28 16:00 【PCメール】10月CP‐BT メール送信/2018-09...,NaN
3,2018-09-29 07:00:13,2006-10-13 00:00:00,NaN,000331001,NaN,NaN,本登録,000331001,NaN,NaN,...,NaN,NaN,NaN,NaN,会社員,NaN,NaN,PCメール,2018-09-28 16:00 【PCメール】10月CP‐BT メール送信/2018-09...,REDM12.10 REDM13.03c<CRLF>002332002<CRLF><CRLF...
4,2018-09-29 07:00:13,2007-08-20 00:00:00,NaN,000431039,NaN,NaN,本登録,000431039,NaN,NaN,...,郵送,NaN,NaN,NaN,会社員,NaN,NaN,NaN,2018-09-28 19:00 【PCメール】10月CP‐BT メール開封/2018-09...,NaN


In [127]:
## 1.1. データ作成の基準となるデータセット(顧客ID)を作成

df_id = df_cust['顧客ID']
df_id[:5]

0    000112093
1    000121059
2    000321123
3    000331001
4    000431039
Name: 顧客ID, dtype: object

In [128]:
## 1.2. 分析対象データを抽出

df_cust = df_cust[['顧客ID','生年月日','性別','携帯TEL','自宅TEL','携帯メール','PCメール','職業']]
df_cust[:5]

,顧客ID,生年月日,性別,携帯TEL,自宅TEL,携帯メール,PCメール,職業
0,000112093,1982-02-10,女,有,無,無,有,会社員
1,000121059,1983-08-06,女,有,有,有,有,会社員
2,000321123,1980-08-04,女,有,有,無,有,会社員
3,000331001,1969-10-17,女,有,有,無,有,会社員
4,000431039,1965-09-20,男,有,無,無,有,会社員


In [129]:
## 2. cancelデータ読み込み

df_cancel = pd.read_csv('./data/org/20181005.cancel.csv', encoding='Shift_JISx0213')
df_cancel[:5]

,更新日時,登録日時,状況,コード,希望日（始）,希望日（終）,希望時刻（始）,希望時刻（終）,希望曜日,施術時間,...,単位,売上区分,商品区分,施術区分,変更区分,汎用備考,認知備考,連絡備考,予約備考,端末コード
0,2018-09-23 16:35:14,2018-09-23 16:33:46,未対応,2000006230,2018-11-03,2018-12-24,11:00,20:30,日/土/祝,75.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,土日で志づ塚さん指名,NaN,NaN
1,2018-09-19 07:53:34,2018-09-19 07:53:34,未確認,53000004495,2018-09-19,2018-10-19,12:00,15:00,日/月/火/水/木/金/土/祝,120.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,（希望メニュー）<CRLF>ホワイトエッセンスクリーニング<CRLF>オーラルリフレクソロジ...,Mozilla/5.0 (iPhone; CPU iPhone OS 11_4_1 like...
2,2018-09-08 22:11:03,2018-09-08 22:11:03,未対応,53000004464,2018-09-22,2018-09-22,19:00,20:00,土,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RCT,NaN,NaN
3,2018-09-08 22:11:20,2018-09-08 22:11:20,未対応,53000004465,2018-10-27,2018-10-27,19:00,20:00,土,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RCT,NaN,NaN
4,2018-09-04 14:15:54,2018-09-04 14:15:54,未対応,2000006142,2018-09-23,2018-09-30,NaN,NaN,日/土/祝,30.0,...,NaN,WEケア,単品,W系,可,NaN,NaN,"9/4ＣＣ<CRLF>9/23,24,30 時間指定なし 指名なし<CRLF>ＭＷ希望",NaN,NaN


In [130]:
## 2.1. キャンセル回数をカウント

df_cancel = df_cancel['顧客ID'].value_counts(dropna=False)
df_cancel[:5]

002212978    7
002752281    3
014512275    3
002152059    3
138120029    3
Name: 顧客ID, dtype: int64

In [131]:
## 2.2. SeriesをDataFrameに変換

df_cancel = df_cancel.to_frame(name='キャンセル回数')
## 結合に備えてindexを顧客ID列に変換
df_cancel = df_cancel.reset_index()
df_cancel.columns = ['顧客ID','キャンセル回数']
df_cancel[:5]

,顧客ID,キャンセル回数
0,002212978,7
1,002752281,3
2,014512275,3
3,002152059,3
4,138120029,3


In [132]:
## 3. contactデータ読み込み

df_contact = pd.read_csv('./data/org/20181005.contact.csv', encoding='Shift_JISx0213')
df_contact[:5]

,更新日時,登録日時,状況,コード,受付日,受付時間,登録店舗,対応店舗,共有店舗,登録担当,...,表示区分,顧客ID,性別,新規区分,当日対応,内容,受付時対応,対応内容,確認備考,端末コード
0,2018-09-04 12:52:29,2018-09-03 11:27:58,対応済,177228,2018-09-03,11:27,本部,渋谷,NaN,CD派遣1,...,店舗,002112136,女,RE,NaN,9/4 10:30〜SW・HJ購入<CRLF><CRLF>青山希望でしたが空きなし<CRLF...,NaN,NaN,NaN,NaN
1,2018-09-29 14:28:30,2018-09-28 13:11:21,対応済,179372,2018-09-28,13:11,本部,青山,NaN,派遣5,...,店舗,002112136,女,RE,NaN,10/3 10:00〜ＳＷ 指名なし<CRLF><CRLF>青山院指定で、10/3の早い時間...,NaN,NaN,NaN,NaN
2,2018-09-25 13:19:56,2018-09-25 13:08:02,未対応,179021,2018-09-25,13:08,本部,福山北,NaN,小野寺唯,...,店舗,002112548,女,RE,要,9/25 14：30〜 ＣＷＥ+ＳＷ<CRLF>銀座Ｒｅ<CRLF>医院直通番号お伝え<CR...,NaN,NaN,NaN,NaN
3,2018-09-21 19:40:21,2018-09-21 11:42:36,対応済,178821,2018-09-21,11:42,本部,成城,NaN,秋山聖果,...,店舗,002121269,女,RE,NaN,9/23 14:00〜Ｃ4-ＣＷＥ 指名なしと仰ってました,NaN,NaN,NaN,NaN
4,2018-09-21 19:40:51,2018-09-21 11:59:45,対応済,178824,2018-09-21,11:59,本部,成城,NaN,秋山聖果,...,店舗,002121269,女,RE,NaN,9/23 14:00〜予約の方<CRLF><CRLF>10/6・7に変更希望でしたがご案内で...,NaN,NaN,NaN,NaN


In [133]:
## 3.1. コンタクト回数をカウント

df_contact = df_contact['顧客ID'].value_counts(dropna=False)
df_contact[:5]

214750005    5
203440006    4
194450007    4
175410024    3
136220034    3
Name: 顧客ID, dtype: int64

In [134]:
## 3.2. SeriesをDataFrameに変換

df_contact = df_contact.to_frame(name='コンタクト回数')
## 結合に備えてindexを顧客ID列に変換
df_contact = df_contact.reset_index()
df_contact.columns = ['顧客ID','コンタクト回数']
df_contact[:5]

,顧客ID,コンタクト回数
0,214750005,5
1,203440006,4
2,194450007,4
3,175410024,3
4,136220034,3


In [135]:
## 4. ctiデータ読み込み

df_cti = pd.read_csv('./data/org/20181005.cti.csv', encoding='Shift_JISx0213')
df_cti[:5]

,更新日時,登録日時,コード,店舗,担当者,電話番号,顧客ID,顧客名
0,2018-09-28 19:17:29,2018-09-28 19:17:29,2041832,2,120005,NaN,001150435,大河原泉
1,2018-09-28 19:18:36,2018-09-28 19:18:36,2041833,2,120005,NaN,001150435,大河原泉
2,2018-09-28 19:20:48,2018-09-28 19:20:48,2041834,2,120005,NaN,001150435,大河原泉
3,2018-09-08 18:39:10,2018-09-08 18:39:10,53040595,53,42,NaN,001150435,大河原泉
4,2018-09-08 18:39:29,2018-09-08 18:39:29,53040596,53,2065,NaN,001150435,大河原泉


In [136]:
## 4.1. 問い合わせ回数をカウント

df_cti = df_cti['顧客ID'].value_counts(dropna=False)
df_cti[:5]

001150435    9
002852369    7
035252216    7
002631152    6
002153174    6
Name: 顧客ID, dtype: int64

In [137]:
## 4.2. SeriesをDataFrameに変換

df_cti = df_cti.to_frame(name='問い合わせ回数')
## 結合に備えてindexを顧客ID列に変換
df_cti = df_cti.reset_index()
df_cti.columns = ['顧客ID','問い合わせ回数']
df_cti[:5]

,顧客ID,問い合わせ回数
0,001150435,9
1,002852369,7
2,035252216,7
3,002631152,6
4,002153174,6


In [138]:
## 5. logデータ読み込み

df_log = pd.read_csv('./data/org/20181005.access_log.csv', encoding='Shift_JISx0213')
df_log[:5]

,IPアドレス,ホスト,アクセス日時,メソッド,パス,HTTPバージョン,ファイル名,ステータスコード,レスポンスバイト数,リファラ,ユーザーエージェント,レスポンスタイム,顧客ID
0,121.102.38.217,www.whiteessence.com,2018-09-02 03:42:41,GET,/reserve/1/,HTTP/1.1,/home/www/www/reserve/1/index.php,200,30414,-,Mozilla/5.0 (iPhone; CPU iPhone OS 11_4_1 like...,274147,063522225
1,113.32.133.185,www.whiteessence.com,2018-09-02 03:45:03,GET,/?utm_source=kuchi-lab&utm_medium=referral&utm...,HTTP/1.1,/home/www/www/sp.php,200,78200,http://kuchi-lab.com/whitening-tokyo-4501,Mozilla/5.0 (iPhone; CPU iPhone OS 11_2 like M...,388227,053122788
2,126.123.46.204,www.whiteessence.com,2018-09-02 03:45:08,GET,/,HTTP/1.1,/home/www/www/sp.php,200,78200,-,Mozilla/5.0 (iPhone; CPU iPhone OS 11_4_1 like...,421633,140220032
3,113.32.133.185,www.whiteessence.com,2018-09-02 03:45:11,GET,/menu/whitening/,HTTP/1.1,/home/www/www/menu/whitening/sp.php,200,69231,https://www.whiteessence.com/?utm_source=kuchi...,Mozilla/5.0 (iPhone; CPU iPhone OS 11_2 like M...,403521,053122788
4,113.32.133.185,www.whiteessence.com,2018-09-02 03:45:26,GET,/menu/whitening/office/,HTTP/1.1,/home/www/www/menu/whitening/office/sp.php,200,71221,https://www.whiteessence.com/menu/whitening/,Mozilla/5.0 (iPhone; CPU iPhone OS 11_2 like M...,357908,053122788


In [139]:
## 5.1. 不要列を削除

df_log = df_log.drop(['IPアドレス','メソッド','パス','HTTPバージョン','ファイル名','レスポンスバイト数','リファラ','ユーザーエージェント','レスポンスタイム'], axis=1)
df_log[:5]

,ホスト,アクセス日時,ステータスコード,顧客ID
0,www.whiteessence.com,2018-09-02 03:42:41,200,063522225
1,www.whiteessence.com,2018-09-02 03:45:03,200,053122788
2,www.whiteessence.com,2018-09-02 03:45:08,200,140220032
3,www.whiteessence.com,2018-09-02 03:45:11,200,053122788
4,www.whiteessence.com,2018-09-02 03:45:26,200,053122788


In [140]:
## 5.2. アクセス日時をunix時間に変更

df_log['アクセス日時_unix'] = pd.to_datetime(df_log['アクセス日時'])
## 不要なアクセス日時列を削除
df_log = df_log.drop(['アクセス日時'],axis=1)
df_log[:5]

,ホスト,ステータスコード,顧客ID,アクセス日時_unix
0,www.whiteessence.com,200,063522225,2018-09-02 03:42:41
1,www.whiteessence.com,200,053122788,2018-09-02 03:45:03
2,www.whiteessence.com,200,140220032,2018-09-02 03:45:08
3,www.whiteessence.com,200,053122788,2018-09-02 03:45:11
4,www.whiteessence.com,200,053122788,2018-09-02 03:45:26


In [141]:
## 5.3. アクセス間隔を計算

## アクセス日時の差(秒)を算出
df_log['アクセス間隔'] = (df_log['アクセス日時_unix'].shift(-1) - df_log['アクセス日時_unix']).dt.seconds
## 顧客IDの同一性を確認
df_log['顧客ID同一当否'] = (df_log['顧客ID'].shift(-1) == df_log['顧客ID'])
## IDが同一でないセルのアクセス間隔をゼロにする
df_log.loc[~df_log['顧客ID同一当否'], 'アクセス間隔'] = 0
# 同一セッションのアクセスであるフラグ
df_log.loc[df_log['顧客ID同一当否'], 'セッションフラグ'] = 1
df_log[:5]

,ホスト,ステータスコード,顧客ID,アクセス日時_unix,アクセス間隔,顧客ID同一当否,セッションフラグ
0,www.whiteessence.com,200,063522225,2018-09-02 03:42:41,0.0,False,NaN
1,www.whiteessence.com,200,053122788,2018-09-02 03:45:03,0.0,False,NaN
2,www.whiteessence.com,200,140220032,2018-09-02 03:45:08,0.0,False,NaN
3,www.whiteessence.com,200,053122788,2018-09-02 03:45:11,15.0,True,1.0
4,www.whiteessence.com,200,053122788,2018-09-02 03:45:26,0.0,False,NaN


In [142]:
# 5.4. 総滞在時間を計算

df_stay_time = df_log.groupby('顧客ID')['アクセス間隔'].sum()
df_stay_time[:5]

顧客ID
000112093    414.0
000121059    511.0
000321123      0.0
000331001    348.0
000431039     14.0
Name: アクセス間隔, dtype: float64

In [143]:
## 5.5. SeriesをDataFrameに変換

df_stay_time = df_stay_time.to_frame(name='アクセス間隔')
## 結合に備えてindexを顧客ID列に変換
df_stay_time = df_stay_time.reset_index()
df_stay_time.columns = ['顧客ID','アクセス間隔']
df_stay_time[:5]

,顧客ID,アクセス間隔
0,000112093,414.0
1,000121059,511.0
2,000321123,0.0
3,000331001,348.0
4,000431039,14.0


In [144]:
## 5.6. df_stay_timeと競合するのでdf_logのアクセス間隔列を削除

df_log = df_log.drop(['アクセス間隔'], axis=1)
df_log[:5]

,ホスト,ステータスコード,顧客ID,アクセス日時_unix,顧客ID同一当否,セッションフラグ
0,www.whiteessence.com,200,063522225,2018-09-02 03:42:41,False,NaN
1,www.whiteessence.com,200,053122788,2018-09-02 03:45:03,False,NaN
2,www.whiteessence.com,200,140220032,2018-09-02 03:45:08,False,NaN
3,www.whiteessence.com,200,053122788,2018-09-02 03:45:11,True,1.0
4,www.whiteessence.com,200,053122788,2018-09-02 03:45:26,False,NaN


In [145]:
## 5.7. 閲覧ページ総数(集計処理)

df_pv = df_log['顧客ID'].value_counts(dropna=False)
df_pv[:5]

140220031    820
025322055    690
006122048    647
057112063    627
002123189    623
Name: 顧客ID, dtype: int64

In [146]:
## 5.8. SeriesをDataFrameに変換

df_pv = df_pv.to_frame(name='閲覧ページ総数')
## 結合に備えてindexを顧客ID列に変換
df_pv = df_pv.reset_index()
df_pv.columns = ['顧客ID','閲覧ページ総数']
df_pv[:5]

,顧客ID,閲覧ページ総数
0,140220031,820
1,025322055,690
2,006122048,647
3,057112063,627
4,002123189,623


In [147]:
## 5.9. セッション時間

df_session = df_log.groupby('顧客ID')['セッションフラグ'].sum()
df_session[:5]

顧客ID
000112093    61.0
000121059    76.0
000321123     0.0
000331001    90.0
000431039     2.0
Name: セッションフラグ, dtype: float64

In [148]:
## 5.10. SeriesをDataFrameに変換

df_session = df_session.to_frame(name='セッション時間')
## 結合に備えてindexを顧客ID列に変換
df_session = df_session.reset_index()
df_session.columns = ['顧客ID','セッション時間']
df_session[:5]

,顧客ID,セッション時間
0,000112093,61.0
1,000121059,76.0
2,000321123,0.0
3,000331001,90.0
4,000431039,2.0


In [149]:
## 6. reserveデータ読み込み

df_reserve = pd.read_csv('./data/org/20181005.reserve.csv', encoding='Shift_JISx0213')
df_reserve[:5]

,更新日時,登録日時,状況,コード,予約日,開始時刻,終了時刻,顧客ID,店舗,施術担当,...,単位,売上区分,商品区分,施術区分,変更区分,汎用備考,媒体備考,確認備考,予約備考,端末コード
0,2018-09-09 12:46:10,2018-09-08 17:39:53,予確済,143000026578,2018-09-09,12:30,13:30,000112093,渋谷,相原美咲,...,NaN,WEケア,単品,C系,可,NaN,NaN,NaN,（希望メニュー）<CRLF>ホワイトエッセンスクリーニング,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....
1,2018-09-24 09:47:19,2018-09-12 21:45:40,予確済,38000055672,2018-09-24,11:00,12:30,000121059,青山,中丸杏子,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,（希望メニュー）<CRLF>ホワイトエッセンスクリーニング<CRLF>歯ぐきマッサージ<CR...,Mozilla/5.0 (iPhone; CPU iPhone OS 11_4_1 like...
2,2018-09-18 08:19:30,2018-09-14 12:30:16,予確済,70000044907,2018-09-18,18:00,19:00,000321123,御茶ノ水,加藤優里,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H30/09/14 お子様が熱が出てキャンセル<CRLF>H29/11/17 にオプションN...,NaN,NaN
3,2018-09-18 19:37:44,2018-09-18 19:00:17,予約,70000044952,2018-12-17,18:00,19:00,000321123,御茶ノ水,加藤優里,...,NaN,WEケア,単品,C系,可,NaN,NaN,C4再,NaN,NaN
4,2018-09-18 19:02:36,2018-09-18 19:02:36,予約,70000044954,2018-10-17,18:00,18:30,000321123,御茶ノ水,太田和秀,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [150]:
## 6.1. 予約回数を集計

df_reserve = df_reserve.groupby(['顧客ID']).size()
df_reserve[:5]

顧客ID
000112093    1
000121059    1
000321123    3
000331001    1
000512107    1
dtype: int64

In [151]:
## 6.2. SeriesをDataFrameに変換

df_reserve = df_reserve.to_frame(name='予約回数')
## 結合に備えてindexを顧客ID列に変換
df_reserve = df_reserve.reset_index()
df_reserve.columns = ['顧客ID','予約回数']
df_reserve[:5]

,顧客ID,予約回数
0,000112093,1
1,000121059,1
2,000321123,3
3,000331001,1
4,000512107,1


In [152]:
## 7. salesデータを読み込み
# 注意）salesデータは顧客IDを主keyにすることはできない

df_sales = pd.read_csv('./data/org/20181005.sales.csv', encoding='Shift_JISx0213')
df_sales[:5]

,更新日時,登録日時,状況,伝票コード,明細コード,伝票区分,会計日,締日,来店日,開始時刻,...,商品名,売上単価,請求単価,数量,単位,売上区分,商品区分,変更区分,明細備考,汎用備考
0,2018-09-09 14:06:55,2018-09-09 08:49:22,確定済,143000022090,1,通常,2018-09-09,2018-09-09,2018-09-09,12:30,...,CWE,8640,8640,1,NaN,WEケア,単品,可,NaN,NaN
1,2018-09-09 14:06:55,2018-09-09 08:49:22,確定済,143000022090,2,通常,2018-09-09,2018-09-09,2018-09-09,12:30,...,OP-ナノアパ500,540,540,1,NaN,WEケア,オプション,可,NaN,NaN
2,2018-09-09 14:06:55,2018-09-09 08:49:22,確定済,143000022090,3,通常,2018-09-09,2018-09-09,2018-09-09,12:30,...,ｾﾗｼﾞｪﾙ,1944,1944,1,NaN,WE物販,物販,可,NaN,NaN
3,2018-09-24 12:20:42,2018-09-24 09:47:20,確定済,38000032362,1,通常,2018-09-24,2018-09-24,2018-09-24,11:00,...,CWE,8640,8640,1,NaN,WEケア,単品,可,NaN,NaN
4,2018-09-24 12:20:42,2018-09-24 09:47:20,確定済,38000032362,2,通常,2018-09-24,2018-09-24,2018-09-24,11:00,...,OP-GM,3240,3240,1,NaN,WEケア,オプション,可,NaN,NaN


In [153]:
## 売上列を追加
df_sales['売上'] = df_sales['売上単価'] * df_sales['数量']

## 個別商品の売上を特定する列を追加
df_sales['明細ID'] = df_sales['伝票コード'] * 10 + df_sales['明細コード']

df_sales[:5]

,更新日時,登録日時,状況,伝票コード,明細コード,伝票区分,会計日,締日,来店日,開始時刻,...,請求単価,数量,単位,売上区分,商品区分,変更区分,明細備考,汎用備考,売上,明細ID
0,2018-09-09 14:06:55,2018-09-09 08:49:22,確定済,143000022090,1,通常,2018-09-09,2018-09-09,2018-09-09,12:30,...,8640,1,NaN,WEケア,単品,可,NaN,NaN,8640,1430000220901
1,2018-09-09 14:06:55,2018-09-09 08:49:22,確定済,143000022090,2,通常,2018-09-09,2018-09-09,2018-09-09,12:30,...,540,1,NaN,WEケア,オプション,可,NaN,NaN,540,1430000220902
2,2018-09-09 14:06:55,2018-09-09 08:49:22,確定済,143000022090,3,通常,2018-09-09,2018-09-09,2018-09-09,12:30,...,1944,1,NaN,WE物販,物販,可,NaN,NaN,1944,1430000220903
3,2018-09-24 12:20:42,2018-09-24 09:47:20,確定済,38000032362,1,通常,2018-09-24,2018-09-24,2018-09-24,11:00,...,8640,1,NaN,WEケア,単品,可,NaN,NaN,8640,380000323621
4,2018-09-24 12:20:42,2018-09-24 09:47:20,確定済,38000032362,2,通常,2018-09-24,2018-09-24,2018-09-24,11:00,...,3240,1,NaN,WEケア,オプション,可,NaN,NaN,3240,380000323622


In [154]:
## 7.2. 顧客属性取得

## 売上1回あたりの情報が欲しいので、明細コード=1のレコードを抽出
## salesは売上1回の中で複数の商品が買われた場合、顧客属性情報が重複して格納される

## 明細コード=1のレコードを中出
df_sales_attr = df_sales.query('明細コード == 1')

## 顧客属性関連列を抽出
df_sales_attr = df_sales_attr[['顧客ID','指名回数','コース受諾回数','紹介カード受渡回数','治療送客回数','院長挨拶回数']]

df_sales_attr[:5]

,顧客ID,指名回数,コース受諾回数,紹介カード受渡回数,治療送客回数,院長挨拶回数
0,112093,なし,なし,あり,なし,あり
3,121059,なし,なし,なし,なし,なし
6,321123,なし,なし,なし,なし,なし
10,331001,なし,なし,なし,なし,なし
14,431039,あり,なし,なし,なし,なし


In [155]:
## 8.1. ファイル結合1

## 顧客属性
cust_attr = pd.merge(df_cust, df_cancel, on='顧客ID', how='left')
cust_attr = pd.merge(cust_attr, df_contact, on='顧客ID', how='left')
cust_attr = pd.merge(cust_attr, df_cti, on='顧客ID', how='left')
cust_attr = pd.merge(cust_attr, df_pv, on='顧客ID', how='left')
cust_attr = pd.merge(cust_attr, df_session, on='顧客ID', how='left')
cust_attr = pd.merge(cust_attr, df_reserve, on='顧客ID', how='left')

cust_attr[:5]

,顧客ID,生年月日,性別,携帯TEL,自宅TEL,携帯メール,PCメール,職業,キャンセル回数,コンタクト回数,問い合わせ回数,閲覧ページ総数,セッション時間,予約回数
0,000112093,1982-02-10,女,有,無,無,有,会社員,NaN,NaN,NaN,82,61.0,1.0
1,000121059,1983-08-06,女,有,有,有,有,会社員,NaN,NaN,NaN,103,76.0,1.0
2,000321123,1980-08-04,女,有,有,無,有,会社員,NaN,NaN,NaN,1,0.0,3.0
3,000331001,1969-10-17,女,有,有,無,有,会社員,NaN,NaN,NaN,142,90.0,1.0
4,000431039,1965-09-20,男,有,無,無,有,会社員,NaN,NaN,NaN,4,2.0,NaN


In [156]:
## 8.2. ファイル結合2

## 売上属性
#sales_attr = pd.merge(df_pv, df_session, on='顧客ID', how='left')
#sales_attr = pd.merge(sales_attr, df_reserve, on='顧客ID', how='left')
#sales_attr = pd.merge(sales_attr, df_sales_attr, on='顧客ID', how='left')

#sales_attr[:5]

In [157]:
## 9. ファイルエクスポート

cust_attr.to_csv('./data/jupyter/cust_attr.csv')
#sales_attr.to_csv('./data/jupyter/sales_attr.csv')
#df_log.to_csv('./data/jupyter/log.csv')
df_sales.to_csv('./data/jupyter/sales.csv')